#### **Install the latest semantic link labs package**

Check [here](https://pypi.org/project/semantic-link-labs/) to see the latest version.

In [ ]:
%pip install semantic-link-labs --q

#### **Import the library and set initial parameters**

In [ ]:
import sempy_labs as fabric
from sempy_labs import ConnectWarehouse
import base64
import pandas as pd
import sempy.fabric as fabric2
from itertools import chain, repeat
import pyodbc
import struct
import sqlalchemy
import time

# set parameters
CDMstorageAccount = None #Enter abfss of CDM container with CSV files
CDMlakehouse_name = None #Enter the lakehouse to create shortcut from

#Instantiate the client
client = fabric2.FabricRestClient()

#Function to create OAuth Bearer Token for WH
def create_engine(connection_string : str):
    token = mssparkutils.credentials.getToken('https://analysis.windows.net/powerbi/api').encode("UTF-16-LE")    
    token_struct = struct.pack(f'<I{len(token)}s', len(token), token)
    SQL_COPT_SS_ACCESS_TOKEN = 1256
    return sqlalchemy.create_engine("mssql+pyodbc://", creator=lambda: pyodbc.connect(connection_string, attrs_before={SQL_COPT_SS_ACCESS_TOKEN: token_struct}))

#### **Notebooks**

**NB: SparkSessionKeepAlive**

In [ ]:
# SparkSessionKeepAlive
nb_name = "SparkSessionKeepAlive"
nb_description = "Keep a Spark Session alive"
nb_payload = "eyJjZWxscyI6W3siY2VsbF90eXBlIjoiY29kZSIsInNvdXJjZSI6W10sIm91dHB1dHMiOltdLCJleGVjdXRpb25fY291bnQiOm51bGwsIm1ldGFkYXRhIjp7Im1pY3Jvc29mdCI6eyJsYW5ndWFnZSI6InB5dGhvbiIsImxhbmd1YWdlX2dyb3VwIjoic3luYXBzZV9weXNwYXJrIn19LCJpZCI6IjM3OGU4YmM1LTNhZmYtNGUzNi04MzA5LTUyNjZmYTUxNDgxOSJ9XSwibWV0YWRhdGEiOnsia2VybmVsX2luZm8iOnsibmFtZSI6InN5bmFwc2VfcHlzcGFyayJ9LCJrZXJuZWxzcGVjIjp7Im5hbWUiOiJzeW5hcHNlX3B5c3BhcmsiLCJkaXNwbGF5X25hbWUiOiJzeW5hcHNlX3B5c3BhcmsifSwibGFuZ3VhZ2VfaW5mbyI6eyJuYW1lIjoicHl0aG9uIn0sIm1pY3Jvc29mdCI6eyJsYW5ndWFnZSI6InB5dGhvbiIsImxhbmd1YWdlX2dyb3VwIjoic3luYXBzZV9weXNwYXJrIiwibXNfc3BlbGxfY2hlY2siOnsibXNfc3BlbGxfY2hlY2tfbGFuZ3VhZ2UiOiJlbiJ9fSwibnRlcmFjdCI6eyJ2ZXJzaW9uIjoibnRlcmFjdC1mcm9udC1lbmRAMS4wLjAifSwic3BhcmtfY29tcHV0ZSI6eyJjb21wdXRlX2lkIjoiL3RyaWRlbnQvZGVmYXVsdCIsInNlc3Npb25fb3B0aW9ucyI6eyJjb25mIjp7InNwYXJrLnN5bmFwc2UubmJzLnNlc3Npb24udGltZW91dCI6IjEyMDAwMDAifX19LCJkZXBlbmRlbmNpZXMiOnsibGFrZWhvdXNlIjp7ImRlZmF1bHRfbGFrZWhvdXNlIjoiODJlMTY4NjgtMWExYS00YzMzLTg1OWYtODcyZjkzNjMxNWFkIiwiZGVmYXVsdF9sYWtlaG91c2VfbmFtZSI6ImxoX0ZuTyIsImRlZmF1bHRfbGFrZWhvdXNlX3dvcmtzcGFjZV9pZCI6ImExYmI1OGY3LWJiYTUtNDc5MS04NWQyLTk0NWY3OTk3MmQxYyJ9LCJlbnZpcm9ubWVudCI6eyJlbnZpcm9ubWVudElkIjoiNWU3NGQwODYtZThhZS00YjVhLWI2YTctZWY2ZWYyZjQ4YWQ3Iiwid29ya3NwYWNlSWQiOiJhMWJiNThmNy1iYmE1LTQ3OTEtODVkMi05NDVmNzk5NzJkMWMifX19LCJuYmZvcm1hdCI6NCwibmJmb3JtYXRfbWlub3IiOjV9"
nb_payloadBase64 = base64.b64decode(nb_payload.encode("ascii")).decode("ascii").encode('utf-8')
fabric.create_notebook(name=nb_name,description=nb_description, notebook_content=nb_payloadBase64)

**NB: CDMtoLh**

In [ ]:
# CDMtoLh
nb_name = "CDMtoLh"
nb_description = "Transformation Notebook from CDM to delta parquet"
nb_payload = "..."
nb_payloadBase64 = base64.b64decode(nb_payload.encode("ascii")).decode("ascii").encode('utf-8')
fabric.create_notebook(name=nb_name,description=nb_description, notebook_content=nb_payloadBase64)

**NB: InitCDMtoLh**

In [ ]:
# InitCDMtoLh
nb_name = "InitCDMtoLh"
nb_description = "Initial Transformation Notebook from CDM to delta parquet for given table name"
nb_payload = "eyJjZWxscyI6W3siY2VsbF90eXBlIjoiY29kZSIsInNvdXJjZSI6WyJjb250YWluZXJQYXRoID0gXCJGaWxlcy9kYXRhdmVyc2UtbTM2NWRldnRzdGRlLXVucTkwNTc0MWZhZjY4MzRiYTZiZGRiYWJkOWQyZGJjXCJcbiIsImFiZnNzUGF0aCA9IFwiYWJmc3M6Ly9EYXRhdmVyc2VQb0NAb25lbGFrZS5kZnMuZmFicmljLm1pY3Jvc29mdC5jb20vTEgyLkxha2Vob3VzZVwiXG4iLCJ0YWJsZW5hbWUgPSBcIlwiIl0sIm91dHB1dHMiOlt7Im91dHB1dF90eXBlIjoiZGlzcGxheV9kYXRhIiwiZGF0YSI6eyJhcHBsaWNhdGlvbi92bmQubGl2eS5zdGF0ZW1lbnQtbWV0YStqc29uIjp7InNwYXJrX3Bvb2wiOm51bGwsInN0YXRlbWVudF9pZCI6MTEsInN0YXRlbWVudF9pZHMiOlsxMV0sInN0YXRlIjoiZmluaXNoZWQiLCJsaXZ5X3N0YXRlbWVudF9zdGF0ZSI6ImF2YWlsYWJsZSIsInNlc3Npb25faWQiOiJlOGIyODVmYy05NzM0LTQ3MzMtOTZhMi03Y2E3ZmJiN2Y3MzMiLCJub3JtYWxpemVkX3N0YXRlIjoiZmluaXNoZWQiLCJxdWV1ZWRfdGltZSI6IjIwMjUtMDEtMTBUMDc6MjQ6MjMuMjgzOTIyNloiLCJzZXNzaW9uX3N0YXJ0X3RpbWUiOm51bGwsImV4ZWN1dGlvbl9zdGFydF90aW1lIjoiMjAyNS0wMS0xMFQwNzoyNDoyMy44MDYzNjA4WiIsImV4ZWN1dGlvbl9maW5pc2hfdGltZSI6IjIwMjUtMDEtMTBUMDc6MjQ6MjQuMDc4NDA3NloiLCJwYXJlbnRfbXNnX2lkIjoiMzAwMzM1M2MtMjc3OC00YzliLTg2NjAtYzIyODMxNjVkODJjIn0sInRleHQvcGxhaW4iOiJTdGF0ZW1lbnRNZXRhKCwgZThiMjg1ZmMtOTczNC00NzMzLTk2YTItN2NhN2ZiYjdmNzMzLCAxMSwgRmluaXNoZWQsIEF2YWlsYWJsZSwgRmluaXNoZWQpIn0sIm1ldGFkYXRhIjp7fX1dLCJleGVjdXRpb25fY291bnQiOjksIm1ldGFkYXRhIjp7Im1pY3Jvc29mdCI6eyJsYW5ndWFnZSI6InB5dGhvbiIsImxhbmd1YWdlX2dyb3VwIjoic3luYXBzZV9weXNwYXJrIn0sInRhZ3MiOlsicGFyYW1ldGVycyJdfSwiaWQiOiJkOTE3ZWVlZC1lM2MyLTRjMWUtYTU5NS0zZDk1NzBhMjlmZTUifSx7ImNlbGxfdHlwZSI6Im1hcmtkb3duIiwic291cmNlIjpbIiMjIyMgKipTY2hlbWEgRnVuY3Rpb25zKioiXSwibWV0YWRhdGEiOnsibnRlcmFjdCI6eyJ0cmFuc2llbnQiOnsiZGVsZXRpbmciOmZhbHNlfX0sIm1pY3Jvc29mdCI6eyJsYW5ndWFnZSI6InB5dGhvbiIsImxhbmd1YWdlX2dyb3VwIjoic3luYXBzZV9weXNwYXJrIn0sImpwLU1hcmtkb3duSGVhZGluZ0NvbGxhcHNlZCI6dHJ1ZX0sImlkIjoiNTAxZTc5NmYtNzNhZS00Njc3LThkMzMtNzE4ZDQzYTZkYWNmIn0seyJjZWxsX3R5cGUiOiJjb2RlIiwic291cmNlIjpbImZyb20gZGVsdGEudGFibGVzIGltcG9ydCAqXG4iLCJmcm9tIHB5c3Bhcmsuc3FsLnR5cGVzIGltcG9ydCAqXG4iLCJmcm9tIHB5c3Bhcmsuc3FsLmZ1bmN0aW9ucyBpbXBvcnQgKlxuIiwiXG4iLCIjIHBhcnNlIENETSBmb3JtYXQgZnJvbSBjc3Ygd2l0aCBtb2RlbC5qc29uIHNjaGVtYSAgXG4iLCJkZWYgZ2V0X3NjaGVtYShjb2xzLGNvbHR5cGVzKTpcbiIsIiAgYXJyID0gW10gIFxuIiwiICBjb2x1bW5zID0gW1wiY29sdW1uX25hbWVcIixcInR5cGVzXCJdIFxuIiwiICAjIHNwZWNpZnkgY29sdW1uIG5hbWVzICAgXG4iLCIgIGNvbGFyciA9IHNwYXJrLmNyZWF0ZURhdGFGcmFtZSh6aXAoY29scywgY29sdHlwZXMpLCBjb2x1bW5zKVxuIiwiICBjb2xtYXAgPSBjb2xhcnIud2l0aENvbHVtbihcImRhdGF0eXBlXCIsIGV4cHIoXCJDQVNFIFdIRU4gdHlwZXMgPSAnc3RyaW5nJyAgICAgVEhFTiAnU3RyaW5nVHlwZSgpJyBcIiArIFxuIiwiICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgXCJXSEVOIGNvbHVtbl9uYW1lID0gJ1NpbmtDcmVhdGVkT24nIFRIRU4gJ1N0cmluZ1R5cGUoKScgXCIgKyBcbiIsIiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIFwiV0hFTiBjb2x1bW5fbmFtZSA9ICdTaW5rTW9kaWZpZWRPbicgVEhFTiAnU3RyaW5nVHlwZSgpJyBcIiArIFxuIiwiICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgXCJXSEVOIHR5cGVzID0gJ2Jvb2xlYW4nICAgIFRIRU4gJ0Jvb2xlYW5UeXBlKCknIFwiICtcbiIsIiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIFwiV0hFTiB0eXBlcyA9ICdkYXRlJyAgICAgICBUSEVOICdEYXRlVHlwZSgpJyBcIiArXG4iLCIgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBcIldIRU4gdHlwZXMgPSAnZGF0ZVRpbWUnICAgVEhFTiAnVGltZXN0YW1wVHlwZSgpJyBcIiArXG4iLCIgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBcIldIRU4gdHlwZXMgPSAnZGVjaW1hbCcgICAgVEhFTiAnRGVjaW1hbFR5cGUoKScgXCIgK1xuIiwiICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgXCJXSEVOIHR5cGVzID0gJ2ludGVnZXInICAgIFRIRU4gJ0ludGVnZXJUeXBlKCknIFwiICtcbiIsIiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIFwiV0hFTiB0eXBlcyA9ICdpbnQ2NCcgICAgICBUSEVOICdJbnRlZ2VyVHlwZSgpJyBcIiArXG4iLCIgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBcIldIRU4gdHlwZXMgPSAnZG91YmxlJyAgICAgVEhFTiAnRG91YmxlVHlwZSgpJyBcIiArXG4iLCIgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBcIldIRU4gdHlwZXMgPSAnZmxvYXQnICAgICAgVEhFTiAnRmxvYXRUeXBlKCknIFwiICtcbiIsIiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIFwiV0hFTiB0eXBlcyA9ICdieXRlJyAgICAgICBUSEVOICdCeXRlVHlwZSgpJyBcIiArICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgXG4iLCIgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBcIkVMU0UgJ1N0cmluZ1R5cGUoKScgRU5EXCIpKS5jb2xsZWN0KClcbiIsIlxuIiwiICBzdHJ1Y3RUeXBlU2NoZW1hID0gU3RydWN0VHlwZShbU3RydWN0RmllbGQoZlswXSwgZXZhbChmWzJdKSwgVHJ1ZSkgZm9yIGYgaW4gY29sbWFwXSkgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBcbiIsIiAgcmV0dXJuIHN0cnVjdFR5cGVTY2hlbWFcbiIsIiAgI2ZvciBjb2wgaW4gY29sbWFwOlxuIiwiICAjICBhcnIuYXBwZW5kKFN0cnVjdEZpZWxkKGNvbC5jb2x1bW5fbmFtZSwgdHlwZShjb2wuZGF0YXR5cGUpLCBUcnVlKSlcbiIsIiAgI3JldHVybiBTdHJ1Y3RUeXBlKGFycilcbiIsIlxuIiwiZGVmIHBvcHVsYXRlX2VudGl0eV9kZih0YWJsZU5hbWUsIHVybCwgY29sdW1uX25hbWVzLCBjb2x1bW5fdHlwZXMpOiAgXG4iLCIgIGZpbGVfcGF0aCA9IHVybCAgXG4iLCIgIGRmID0gKFxuIiwiICAgIHNwYXJrLnJlYWQub3B0aW9uKFwiaGVhZGVyXCIsIFwiZmFsc2VcIilcbiIsIiAgICAuc2NoZW1hKGdldF9zY2hlbWEoY29sdW1uX25hbWVzLGNvbHVtbl90eXBlcykpXG4iLCIgICAgLm9wdGlvbihcImRlbGltaXRlclwiLCAnLCcpXG4iLCIgICAgLm9wdGlvbihcImVzY2FwZVwiLCdcIicpXG4iLCIgICAgIy5vcHRpb24oXCJ0aW1lc3RhbXBGb3JtYXRcIiwgXCJNTS9kZC95eXl5IEhIOm1tOnNzLlNTU1NTUyBhXCIpXG4iLCIgICAgLm9wdGlvbihcInVuZXNjYXBlZFF1b3RlSGFuZGxpbmdcIiwgXCJTVE9QX0FUX0NMT1NJTkdfUVVPVEVcIilcbiIsIiAgICAub3B0aW9uKFwibXVsdGlMaW5lXCIsIFwidHJ1ZVwiKVxuIiwiICAgIC5jc3YoZmlsZV9wYXRoKVxuIiwiICApXG4iLCIgIHJldHVybiBkZi50b0RGKCpjb2x1bW5fbmFtZXMpIl0sIm91dHB1dHMiOlt7Im91dHB1dF90eXBlIjoiZGlzcGxheV9kYXRhIiwiZGF0YSI6eyJhcHBsaWNhdGlvbi92bmQubGl2eS5zdGF0ZW1lbnQtbWV0YStqc29uIjp7InNwYXJrX3Bvb2wiOm51bGwsInN0YXRlbWVudF9pZCI6MTIsInN0YXRlbWVudF9pZHMiOlsxMl0sInN0YXRlIjoiZmluaXNoZWQiLCJsaXZ5X3N0YXRlbWVudF9zdGF0ZSI6ImF2YWlsYWJsZSIsInNlc3Npb25faWQiOiJlOGIyODVmYy05NzM0LTQ3MzMtOTZhMi03Y2E3ZmJiN2Y3MzMiLCJub3JtYWxpemVkX3N0YXRlIjoiZmluaXNoZWQiLCJxdWV1ZWRfdGltZSI6IjIwMjUtMDEtMTBUMDc6MjQ6MjMuNTAzNjAxM1oiLCJzZXNzaW9uX3N0YXJ0X3RpbWUiOm51bGwsImV4ZWN1dGlvbl9zdGFydF90aW1lIjoiMjAyNS0wMS0xMFQwNzoyNDoyNC41NjUyODE4WiIsImV4ZWN1dGlvbl9maW5pc2hfdGltZSI6IjIwMjUtMDEtMTBUMDc6MjQ6MjQuODA5MTkxWiIsInBhcmVudF9tc2dfaWQiOiI1ODdlNWE3My04MjQ3LTQ1ZTgtOGVlYi00MWFlMGJkMzBkMmMifSwidGV4dC9wbGFpbiI6IlN0YXRlbWVudE1ldGEoLCBlOGIyODVmYy05NzM0LTQ3MzMtOTZhMi03Y2E3ZmJiN2Y3MzMsIDEyLCBGaW5pc2hlZCwgQXZhaWxhYmxlLCBGaW5pc2hlZCkifSwibWV0YWRhdGEiOnt9fV0sImV4ZWN1dGlvbl9jb3VudCI6MTAsIm1ldGFkYXRhIjp7Imp1cHl0ZXIiOnsic291cmNlX2hpZGRlbiI6ZmFsc2UsIm91dHB1dHNfaGlkZGVuIjpmYWxzZX0sIm50ZXJhY3QiOnsidHJhbnNpZW50Ijp7ImRlbGV0aW5nIjpmYWxzZX19LCJtaWNyb3NvZnQiOnsibGFuZ3VhZ2UiOiJweXRob24iLCJsYW5ndWFnZV9ncm91cCI6InN5bmFwc2VfcHlzcGFyayJ9fSwiaWQiOiJkOGJlOWFhYS05N2E3LTQxZjQtOTIzYy04MWRhMGEyMDIyMDEifSx7ImNlbGxfdHlwZSI6Im1hcmtkb3duIiwic291cmNlIjpbIiMjIyMgKipNZXJnZSBGdW5jdGlvbnMqKiJdLCJtZXRhZGF0YSI6eyJudGVyYWN0Ijp7InRyYW5zaWVudCI6eyJkZWxldGluZyI6ZmFsc2V9fSwibWljcm9zb2Z0Ijp7Imxhbmd1YWdlIjoicHl0aG9uIiwibGFuZ3VhZ2VfZ3JvdXAiOiJzeW5hcHNlX3B5c3BhcmsifSwianAtTWFya2Rvd25IZWFkaW5nQ29sbGFwc2VkIjp0cnVlfSwiaWQiOiJiYmIxZDg0Zi1iMWNjLTQzOGQtYmVjZS1jZjliZjgyN2UxZjQifSx7ImNlbGxfdHlwZSI6ImNvZGUiLCJzb3VyY2UiOlsiZnJvbSBkZWx0YS50YWJsZXMgaW1wb3J0ICpcbiIsIlxuIiwiIyBTZXQgU2NoZW1hIGV2b2x1dGlvblxuIiwic3BhcmsuY29uZi5zZXQoXCJzcGFyay5kYXRhYnJpY2tzLmRlbHRhLnNjaGVtYS5hdXRvTWVyZ2UuZW5hYmxlZFwiLCBcInRydWVcIilcbiIsIlxuIiwiZGVmIGJ1aWxkX2pvaW5fY29uZGl0aW9uKHNvdXJjZV9hbGlhcywgdGFyZ2V0X2FsaWFzLCBqb2luX2NvbHVtbnMpOlxuIiwiICAgIHJldHVybiBcIiBBTkQgXCIuam9pbihbZlwie3RhcmdldF9hbGlhc30ue2NvbH0gPD0+IHtzb3VyY2VfYWxpYXN9Lntjb2x9XCIgZm9yIGNvbCBpbiBqb2luX2NvbHVtbnNdKVxuIiwiXG4iLCJkZWYgcmVtb3ZlX2l0ZW1zX2Zyb21fbGlzdChpdGVtcywgaXRlbXNfdG9fcmVtb3ZlKTpcbiIsIiAgICByZXR1cm4gW3ZhbHVlIGZvciB2YWx1ZSBpbiBpdGVtcyBpZiB2YWx1ZSBub3QgaW4gaXRlbXNfdG9fcmVtb3ZlXVxuIiwiXG4iLCJkZWYgYnVpbGRfdXBkYXRlX2NvbmRpdGlvbihzb3VyY2VfYWxpYXMsIHRhcmdldF9hbGlhcywgam9pbl9jb2x1bW5zLCBjb2x1bW5zKTpcbiIsIiAgICAjIEZpcnN0IHJlbW92ZSB0aGUgam9pbiBjb2x1bW5zLCBhcyB3ZSBETyBOT1QgdXBkYXRlIGpvaW4gY29sdW1ucyFcbiIsIiAgICBjb2x1bW5zX3RvX3VwZGF0ZSA9IHJlbW92ZV9pdGVtc19mcm9tX2xpc3QoY29sdW1ucywgam9pbl9jb2x1bW5zKVxuIiwiIFxuIiwiICAgICMgTm93IGJ1aWxkIHRoZSB1cGRhdGUgZGlmZmVyZW5jZXMgY2hlY2sgY29uZGl0aW9uXG4iLCIgICAgcmV0dXJuIFwiIE9SIFwiLmpvaW4oW2ZcInt0YXJnZXRfYWxpYXN9Lntjb2x9IDw+IHtzb3VyY2VfYWxpYXN9Lntjb2x9XCIgZm9yIGNvbCBpbiBjb2x1bW5zX3RvX3VwZGF0ZV0pXG4iLCJcbiIsImRlZiBidWlsZF91cGRhdGVfc2V0KHNvdXJjZV9hbGlhcywgdGFyZ2V0X2FsaWFzLCBjb2x1bW5zKTpcbiIsIiAgICByZXR1cm4gZGljdChbKGZcInt0YXJnZXRfYWxpYXN9LntrZXl9XCIsIGZcIntzb3VyY2VfYWxpYXN9Lnt2YWx9XCIpIGZvciBrZXksIHZhbCBpbiB6aXAoY29sdW1uc1s6OjFdLCBjb2x1bW5zWzo6MV0pXSlcbiIsIlxuIiwiZGVmIG1lcmdlX3NvdXJjZV9pbnRvX3RhcmdldChzb3VyY2VfdGFibGVfbmFtZSwgdGFyZ2V0X3RhYmxlX25hbWUsIGpvaW5fY29sdW1ucywgY29sdW1uc190b19leGNsdWRlKTpcbiIsIiAgICAjIFNldCB0aGUgYWxpYXMgbmFtZXMgZm9yIHNvdXJjZSBhbmQgdGFyZ2V0XG4iLCIgICAgc291cmNlX2FsaWFzID0gXCJzb3VyY2VcIlxuIiwiICAgIHRhcmdldF9hbGlhcyA9IFwidGFyZ2V0XCJcbiIsIiBcbiIsIiAgICAjIEdldCB0aGUgc291cmNlIGRhdGEgaW50byBhIGRhdGEgZnJhbWVcbiIsIiAgICBzb3VyY2VfZGYgPSBzcGFyay5yZWFkLnRhYmxlKHNvdXJjZV90YWJsZV9uYW1lKVxuIiwiICAgIHNvdXJjZV9kZiA9IHNvdXJjZV9kZi5kcm9wKCpjb2x1bW5zX3RvX2V4Y2x1ZGUpXG4iLCIgICAgIFxuIiwiICAgICMjIEdldCBhIGxpc3Qgb2YgY29sdW1uIG5hbWVzIGZyb20gdGhlIHNvdXJjZSBkYXRhZnJhbWUsIHRoZW4gcmVtb3ZlIHRoZSBrZXkgZmllbGRzXG4iLCIgICAgIyNjb2x1bW5fbmFtZXMgPSBzb3VyY2VfZGYuc2NoZW1hLm5hbWVzICAgIFxuIiwiICAgICMjY29sdW1uc190b191cGRhdGUgPSByZW1vdmVfaXRlbXNfZnJvbV9saXN0KGNvbHVtbl9uYW1lcywgam9pbl9jb2x1bW5zKSAgICBcbiIsIiBcbiIsIiAgICAjIEJ1aWxkIHRoZSBqb2luIGNvbmRpdGlvblxuIiwiICAgIGpvaW5fY29uZGl0aW9uID0gYnVpbGRfam9pbl9jb25kaXRpb24oc291cmNlX2FsaWFzLCB0YXJnZXRfYWxpYXMsIGpvaW5fY29sdW1ucylcbiIsIiAgICAjcHJpbnQoZlwiTUVSR0UgLSBqb2luIGNvbmRpdGlvbiA9ICd7am9pbl9jb25kaXRpb259J1wiKVxuIiwiICAgICBcbiIsIiAgICAjIyBCdWlsZCB0aGUgdXBkYXRlIGNvbmRpdGlvblxuIiwiICAgICMjdXBkYXRlX2NvbmRpdGlvbiA9IGJ1aWxkX3VwZGF0ZV9jb25kaXRpb24oc291cmNlX2FsaWFzLCB0YXJnZXRfYWxpYXMsIGpvaW5fY29sdW1ucywgY29sdW1uX25hbWVzKVxuIiwiICAgICMjcHJpbnQoZlwiTUVSR0UgLSB1cGRhdGUgY29uZGl0aW9uID0gJ3t1cGRhdGVfY29uZGl0aW9ufSdcIilcbiIsIiBcbiIsIiAgICAjIyBCdWlsZCB0aGUgdXBkYXRlICdzZXQnIGZpZWxkcyAgICBcbiIsIiAgICAjI3VwZGF0ZV9zZXQgPSBidWlsZF91cGRhdGVfc2V0KHNvdXJjZV9hbGlhcywgdGFyZ2V0X2FsaWFzLCByZW1vdmVfaXRlbXNfZnJvbV9saXN0KGNvbHVtbnNfdG9fdXBkYXRlLCBjb2x1bW5zX3RvX2V4Y2x1ZGUpKVxuIiwiICAgICMjcHJpbnQoZlwiTUVSR0UgLSB1cGRhdGUgc2V0ID0gJ3t1cGRhdGVfc2V0fSdcIilcbiIsIiBcbiIsIiAgICBpZiBEZWx0YVRhYmxlLmlzRGVsdGFUYWJsZShzcGFyaywgdGFyZ2V0X3RhYmxlX25hbWUpOlxuIiwiICAgICAgICAjIEdldCB0aGUgdGFyZ2V0IHRhYmxlXG4iLCIgICAgICAgIHRhcmdldF90YWJsZSA9IERlbHRhVGFibGUuZm9yUGF0aChzcGFyaywgdGFyZ2V0X3RhYmxlX25hbWUpICAgICAgICBcbiIsIiAgICBcbiIsIiAgICAgICAgIyBSdW4gdGhlIG1lcmdlXG4iLCIgICAgICAgIHRhcmdldF90YWJsZS5hbGlhcyh0YXJnZXRfYWxpYXMpIFxcXG4iLCIgICAgICAgICAgICAubWVyZ2Uoc291cmNlID0gc291cmNlX2RmLmFsaWFzKHNvdXJjZV9hbGlhcyksIGNvbmRpdGlvbiA9IGpvaW5fY29uZGl0aW9uKSBcXFxuIiwiICAgICAgICAgICAgLndpdGhTY2hlbWFFdm9sdXRpb24oKSBcXFxuIiwiICAgICAgICAgICAgLndoZW5Ob3RNYXRjaGVkSW5zZXJ0QWxsKCkgXFxcbiIsIiAgICAgICAgICAgIC53aGVuTWF0Y2hlZFVwZGF0ZUFsbCgpIFxcXG4iLCIgICAgICAgICAgICAuZXhlY3V0ZSgpXG4iLCIgICAgZWxzZTogICAgICAgXG4iLCIgICAgICAgICMgSW5pdGlhbGl6ZSBUYXJnZXQgYnkgU291cmNlICAgICAgICAgIFxuIiwiICAgICAgICBzb3VyY2VfZGYud3JpdGUubW9kZShcIm92ZXJ3cml0ZVwiKS5vcHRpb24oXCJtZXJnZVNjaGVtYVwiLFwidHJ1ZVwiKS5mb3JtYXQoXCJkZWx0YVwiKS5zYXZlKHRhcmdldF90YWJsZV9uYW1lKVxuIiwiXG4iLCIgICAgcmV0dXJuIDEiXSwib3V0cHV0cyI6W3sib3V0cHV0X3R5cGUiOiJkaXNwbGF5X2RhdGEiLCJkYXRhIjp7ImFwcGxpY2F0aW9uL3ZuZC5saXZ5LnN0YXRlbWVudC1tZXRhK2pzb24iOnsic3BhcmtfcG9vbCI6bnVsbCwic3RhdGVtZW50X2lkIjoxMywic3RhdGVtZW50X2lkcyI6WzEzXSwic3RhdGUiOiJmaW5pc2hlZCIsImxpdnlfc3RhdGVtZW50X3N0YXRlIjoiYXZhaWxhYmxlIiwic2Vzc2lvbl9pZCI6ImU4YjI4NWZjLTk3MzQtNDczMy05NmEyLTdjYTdmYmI3ZjczMyIsIm5vcm1hbGl6ZWRfc3RhdGUiOiJmaW5pc2hlZCIsInF1ZXVlZF90aW1lIjoiMjAyNS0wMS0xMFQwNzoyNDoyMy42MzU0NzMxWiIsInNlc3Npb25fc3RhcnRfdGltZSI6bnVsbCwiZXhlY3V0aW9uX3N0YXJ0X3RpbWUiOiIyMDI1LTAxLTEwVDA3OjI0OjI1LjMzODk1ODVaIiwiZXhlY3V0aW9uX2ZpbmlzaF90aW1lIjoiMjAyNS0wMS0xMFQwNzoyNDoyNS42MDc3MTg0WiIsInBhcmVudF9tc2dfaWQiOiIwYzRlNWJlNy0yZDAyLTQwMTItYjM5YS1kZDBmNjQ0M2NlYTkifSwidGV4dC9wbGFpbiI6IlN0YXRlbWVudE1ldGEoLCBlOGIyODVmYy05NzM0LTQ3MzMtOTZhMi03Y2E3ZmJiN2Y3MzMsIDEzLCBGaW5pc2hlZCwgQXZhaWxhYmxlLCBGaW5pc2hlZCkifSwibWV0YWRhdGEiOnt9fV0sImV4ZWN1dGlvbl9jb3VudCI6MTEsIm1ldGFkYXRhIjp7Imp1cHl0ZXIiOnsic291cmNlX2hpZGRlbiI6ZmFsc2UsIm91dHB1dHNfaGlkZGVuIjpmYWxzZX0sIm50ZXJhY3QiOnsidHJhbnNpZW50Ijp7ImRlbGV0aW5nIjpmYWxzZX19LCJtaWNyb3NvZnQiOnsibGFuZ3VhZ2UiOiJweXRob24iLCJsYW5ndWFnZV9ncm91cCI6InN5bmFwc2VfcHlzcGFyayJ9fSwiaWQiOiIxOWFhMTJhMC1hNDFlLTQwNGEtYWY3MS1kYzY0ZjVkNDk4ZWIifSx7ImNlbGxfdHlwZSI6Im1hcmtkb3duIiwic291cmNlIjpbIiMjIyMgKipTaG9ydCBkZiBsb2FkaW5nIHdpdGggd2lsZGNhcmRzKioiXSwibWV0YWRhdGEiOnsibnRlcmFjdCI6eyJ0cmFuc2llbnQiOnsiZGVsZXRpbmciOmZhbHNlfX0sIm1pY3Jvc29mdCI6eyJsYW5ndWFnZSI6InB5dGhvbiIsImxhbmd1YWdlX2dyb3VwIjoic3luYXBzZV9weXNwYXJrIn19LCJpZCI6ImIzODYxMWUxLTZlYmItNGU1Ny1iOWIxLTI0NGQ5OGM1NjE2ZiJ9LHsiY2VsbF90eXBlIjoiY29kZSIsInNvdXJjZSI6WyJpbXBvcnQgb3NcbiIsImZyb20gcGF0aGxpYiBpbXBvcnQgUGF0aFxuIiwiZnJvbSBweXNwYXJrLnNxbC5mdW5jdGlvbnMgaW1wb3J0IGNvbCwgZXhwclxuIiwiXG4iLCIjIEVuYWJsZSBWLU9yZGVyXG4iLCJzcGFyay5jb25mLnNldChcInNwYXJrLnNxbC5wYXJxdWV0LnZvcmRlci5lbmFibGVkXCIsIFwidHJ1ZVwiKSAgICBcbiIsIiMgRW5hYmxlIGF1dG9tYXRpYyBEZWx0YSBvcHRpbWl6ZWQgd3JpdGVcbiIsInNwYXJrLmNvbmYuc2V0KFwic3BhcmsubWljcm9zb2Z0LmRlbHRhLm9wdGltaXplV3JpdGUuZW5hYmxlZFwiLCBcInRydWVcIilcbiIsIiMgU2V0IGRhdGUvdGltZSBjb21wYXRpYmlsaXR5IG9wdGlvbnNcbiIsInNwYXJrLmNvbmYuc2V0KFwic3Bhcmsuc3FsLnBhcnF1ZXQuaW50OTZSZWJhc2VNb2RlSW5Xcml0ZVwiLCBcIkNPUlJFQ1RFRFwiKVxuIiwic3BhcmsuY29uZi5zZXQoXCJzcGFyay5zcWwubGVnYWN5LnRpbWVQYXJzZXJQb2xpY3lcIiwgXCJDT1JSRUNURURcIilcbiIsIlxuIiwiIyBzZXQgcGF0aCBpbmZvcm1hdGlvblxuIiwiZm9sZGVycGF0aCA9IGFiZnNzUGF0aCArIFwiL1wiICsgY29udGFpbmVyUGF0aCArIFwiL1wiXG4iLCJmaWxlcyA9IG5vdGVib29rdXRpbHMuZnMubHMoZm9sZGVycGF0aClcbiIsIlxuIiwiIyBsb2FkIG1vZGVsLmpzb25cbiIsIm1vZGVsZm9sZGVycyA9IG5vdGVib29rdXRpbHMuZnMubHMoZm9sZGVycGF0aClcbiIsIm1vZGVsZmlsZXMgPSBbbW9kZWxmaWxlLnBhdGggZm9yIG1vZGVsZmlsZSBpbiBtb2RlbGZvbGRlcnMgaWYgbW9kZWxmaWxlLm5hbWUuZW5kc3dpdGgoXCJtb2RlbC5qc29uXCIpXVxuIiwibW9kZWwgPSBzcGFyay5yZWFkLmpzb24oZm9sZGVycGF0aCArIFwiL21vZGVsLmpzb25cIiwgbXVsdGlMaW5lPVRydWUpXG4iLCJlbnRpdGllcyA9IG1vZGVsLnNlbGVjdChleHBsb2RlKG1vZGVsW1wiZW50aXRpZXNcIl0pLmFsaWFzKFwiZW50aXR5XCIpKVxuIiwiZW50aXR5X2luZm8gPSBlbnRpdGllcy5zZWxlY3QoXCJlbnRpdHkubmFtZVwiLCBcImVudGl0eS5hdHRyaWJ1dGVzXCIsIFwiZW50aXR5LnBhcnRpdGlvbnNcIilcbiIsImZpbHRlcmVkX2VudGl0eV9pbmZvID0gZW50aXR5X2luZm8uZmlsdGVyKGNvbChcIm5hbWVcIikgPT0gdGFibGVuYW1lKVxuIiwiZW50aXR5X21ldGFkYXRhID0gKFxuIiwiICAgIGZpbHRlcmVkX2VudGl0eV9pbmZvLndpdGhDb2x1bW4oXCJhdHRyaWJ1dGVzXCIsIGV4cGxvZGUoXCJhdHRyaWJ1dGVzXCIpKVxuIiwiICAgIC5zZWxlY3QoXCJuYW1lXCIsIFwicGFydGl0aW9uc1wiLCBjb2woXCJhdHRyaWJ1dGVzXCIpW1wibmFtZVwiXS5hbGlhcyhcImNvbHVtbl9uYW1lXCIpLCBjb2woXCJhdHRyaWJ1dGVzXCIpW1wiZGF0YXR5cGVcIl0uYWxpYXMoXCJjb2x1bW5fdHlwZVwiKSlcbiIsIiAgICApICAgICAgICAgICBcbiIsImVudGl0eV9tZXRhZGF0YSA9IChcbiIsIiAgICBlbnRpdHlfbWV0YWRhdGEuZ3JvdXBCeShcIm5hbWVcIiwgXCJwYXJ0aXRpb25zXCIpXG4iLCIgICAgLmFnZyhjb2xsZWN0X2xpc3QoXCJjb2x1bW5fbmFtZVwiKS5hbGlhcyhcImNvbHVtbnNcIiksY29sbGVjdF9saXN0KFwiY29sdW1uX3R5cGVcIikuYWxpYXMoXCJ0eXBlc1wiKSkgICAgICAgICAgICAgICAgXG4iLCIgICAgLnNlbGVjdChcIipcIilcbiIsIiAgICApXG4iLCJcbiIsIiMgY29sbGVjdCBtZXRhZGF0YSBmb3Igc2NoZW1hXG4iLCJhcnJheV9vZl9tZXRhZGF0YXMgPSBlbnRpdHlfbWV0YWRhdGEuY29sbGVjdCgpXG4iLCJjZG1fbWV0YWRhdGEgPSBuZXh0KHggZm9yIHggaW4gYXJyYXlfb2ZfbWV0YWRhdGFzIGlmIHgubmFtZSA9PSB0YWJsZW5hbWUpXG4iLCJcbiIsIiMgUmVhZCB3aWxkY2FyZCBjc3YgZmlsZXNcbiIsImNkbV9kZiA9IHBvcHVsYXRlX2VudGl0eV9kZihjZG1fbWV0YWRhdGEubmFtZSwgZm9sZGVycGF0aCArIFwiLyovXCIrIHRhYmxlbmFtZSArIFwiLyouY3N2XCIgLCBjZG1fbWV0YWRhdGEuY29sdW1ucyAsIGNkbV9tZXRhZGF0YS50eXBlcykgICAgXG4iLCJcbiIsIiMjIyBTdG9yZSBVcGRhdGUgZGYgaW50byBVcGRhdGUgdGFibGVcbiIsInRhcmdldFRhYmxlTmFtZSA9IHRhYmxlbmFtZVxuIiwic291cmNlVGFibGVOYW1lID0gXCJ1cGRhdGVfXCIgKyB0YWJsZW5hbWUgXG4iLCIjIEhhbmRsZSBkYXRldGltZSBmb3JtYXQgZm9yIHNpbmsgZmllbGRzXG4iLCJjZG1fZGYgPSBjZG1fZGYud2l0aENvbHVtbihcIlNpbmtNb2RpZmllZE9uXCIsdG9fdGltZXN0YW1wKGNvbChcIlNpbmtNb2RpZmllZE9uXCIpLFwiTS9kL3l5eXkgaDptbTpzcyBhXCIpKSBcXFxuIiwiICAgICAgICAgICAgICAgLndpdGhDb2x1bW4oXCJTaW5rQ3JlYXRlZE9uXCIsdG9fdGltZXN0YW1wKGNvbChcIlNpbmtDcmVhdGVkT25cIiksXCJNL2QveXl5eSBoOm1tOnNzIGFcIikpXG4iLCJjZG1fZGYuY3JlYXRlT3JSZXBsYWNlVGVtcFZpZXcoc291cmNlVGFibGVOYW1lKVxuIiwiY2RtX2RmMiA9IHNwYXJrLnNxbChmXCJcIlwiU0VMRUNUICogRlJPTSAoU0VMRUNUICoscm93X251bWJlcigpIE9WRVIgKFBBUlRJVElPTiBCWSBJZCxJc0RlbGV0ZSBPUkRFUiBCWSBTaW5rTW9kaWZpZWRPbiBERVNDKSBBUyByYW5rIEZST00ge3NvdXJjZVRhYmxlTmFtZX0pIFdIRVJFIHJhbmsgPSAxIE9SREVSIEJZIElkLCBTaW5rTW9kaWZpZWRPbiBERVNDXCJcIlwiKVxuIiwiIyByZW1vdmUgcmFua1xuIiwiY2RtX2RmMyA9IGNkbV9kZjIuZHJvcCgncmFuaycpXG4iLCJjZG1fZGYzLmNyZWF0ZU9yUmVwbGFjZVRlbXBWaWV3KHNvdXJjZVRhYmxlTmFtZSlcbiIsIlxuIiwiIyMjIE1lcmdlIGludG8gU2lsdmVyIGZyb20gVXBkYXRlIHRhYmxlIHVzaW5nIFNDRCAyXG4iLCJtZXJnZV9zb3VyY2VfaW50b190YXJnZXQoc291cmNlVGFibGVOYW1lLFwiVGFibGVzL1wiICsgdGFyZ2V0VGFibGVOYW1lLFtcIklkXCIsXCJJc0RlbGV0ZVwiXSxbXCJyYW5rXCJdKSJdLCJvdXRwdXRzIjpbeyJvdXRwdXRfdHlwZSI6ImRpc3BsYXlfZGF0YSIsImRhdGEiOnsiYXBwbGljYXRpb24vdm5kLmxpdnkuc3RhdGVtZW50LW1ldGEranNvbiI6eyJzcGFya19wb29sIjpudWxsLCJzdGF0ZW1lbnRfaWQiOjExLCJzdGF0ZW1lbnRfaWRzIjpbMTFdLCJzdGF0ZSI6ImZpbmlzaGVkIiwibGl2eV9zdGF0ZW1lbnRfc3RhdGUiOiJhdmFpbGFibGUiLCJzZXNzaW9uX2lkIjoiODQwMDYxYTMtMTIwNy00ODRmLWE0MjgtYTRlYjZlY2QwZTUxIiwibm9ybWFsaXplZF9zdGF0ZSI6ImZpbmlzaGVkIiwicXVldWVkX3RpbWUiOiIyMDI1LTAxLTA4VDE2OjMwOjQxLjc2Njk5ODZaIiwic2Vzc2lvbl9zdGFydF90aW1lIjpudWxsLCJleGVjdXRpb25fc3RhcnRfdGltZSI6IjIwMjUtMDEtMDhUMTY6MzA6NDIuMTg0Mzc0WiIsImV4ZWN1dGlvbl9maW5pc2hfdGltZSI6IjIwMjUtMDEtMDhUMTY6MzA6NTYuNzM2ODUyNloiLCJwYXJlbnRfbXNnX2lkIjoiYTc2OTA2MDctZmI2Yi00NmYxLTk0YzEtNTliMGI3MmE5NGM5In0sInRleHQvcGxhaW4iOiJTdGF0ZW1lbnRNZXRhKCwgODQwMDYxYTMtMTIwNy00ODRmLWE0MjgtYTRlYjZlY2QwZTUxLCAxMSwgRmluaXNoZWQsIEF2YWlsYWJsZSwgRmluaXNoZWQpIn0sIm1ldGFkYXRhIjp7fX0seyJvdXRwdXRfdHlwZSI6ImV4ZWN1dGVfcmVzdWx0IiwiZXhlY3V0aW9uX2NvdW50IjozMiwiZGF0YSI6eyJ0ZXh0L3BsYWluIjoiMSJ9LCJtZXRhZGF0YSI6e319XSwiZXhlY3V0aW9uX2NvdW50Ijo5LCJtZXRhZGF0YSI6eyJqdXB5dGVyIjp7InNvdXJjZV9oaWRkZW4iOmZhbHNlLCJvdXRwdXRzX2hpZGRlbiI6ZmFsc2V9LCJudGVyYWN0Ijp7InRyYW5zaWVudCI6eyJkZWxldGluZyI6ZmFsc2V9fSwibWljcm9zb2Z0Ijp7Imxhbmd1YWdlIjoicHl0aG9uIiwibGFuZ3VhZ2VfZ3JvdXAiOiJzeW5hcHNlX3B5c3BhcmsifSwiZWRpdGFibGUiOnRydWUsInJ1bl9jb250cm9sIjp7ImZyb3plbiI6ZmFsc2V9fSwiaWQiOiIxNmMwMWMxNS0wYjdhLTQwYjUtYTVjMS03OTcyYmE3NDFkYzkifV0sIm1ldGFkYXRhIjp7Imtlcm5lbF9pbmZvIjp7Im5hbWUiOiJzeW5hcHNlX3B5c3BhcmsifSwia2VybmVsc3BlYyI6eyJuYW1lIjoic3luYXBzZV9weXNwYXJrIiwibGFuZ3VhZ2UiOiJQeXRob24iLCJkaXNwbGF5X25hbWUiOiJTeW5hcHNlIFB5U3BhcmsifSwibGFuZ3VhZ2VfaW5mbyI6eyJuYW1lIjoicHl0aG9uIn0sIm1pY3Jvc29mdCI6eyJsYW5ndWFnZSI6InB5dGhvbiIsImxhbmd1YWdlX2dyb3VwIjoic3luYXBzZV9weXNwYXJrIiwibXNfc3BlbGxfY2hlY2siOnsibXNfc3BlbGxfY2hlY2tfbGFuZ3VhZ2UiOiJlbiJ9fSwibnRlcmFjdCI6eyJ2ZXJzaW9uIjoibnRlcmFjdC1mcm9udC1lbmRAMS4wLjAifSwid2lkZ2V0cyI6e30sInN5bmFwc2Vfd2lkZ2V0Ijp7InZlcnNpb24iOiIwLjEiLCJzdGF0ZSI6e319LCJzcGFya19jb21wdXRlIjp7ImNvbXB1dGVfaWQiOiIvdHJpZGVudC9kZWZhdWx0Iiwic2Vzc2lvbl9vcHRpb25zIjp7ImNvbmYiOnsic3Bhcmsuc3luYXBzZS5uYnMuc2Vzc2lvbi50aW1lb3V0IjoiMTIwMDAwMCJ9fX0sImRlcGVuZGVuY2llcyI6eyJsYWtlaG91c2UiOnsiZGVmYXVsdF9sYWtlaG91c2UiOiI4MmUxNjg2OC0xYTFhLTRjMzMtODU5Zi04NzJmOTM2MzE1YWQiLCJkZWZhdWx0X2xha2Vob3VzZV9uYW1lIjoiTEgyIiwiZGVmYXVsdF9sYWtlaG91c2Vfd29ya3NwYWNlX2lkIjoiYTFiYjU4ZjctYmJhNS00NzkxLTg1ZDItOTQ1Zjc5OTcyZDFjIiwia25vd25fbGFrZWhvdXNlcyI6W3siaWQiOiI4MmUxNjg2OC0xYTFhLTRjMzMtODU5Zi04NzJmOTM2MzE1YWQifV19LCJlbnZpcm9ubWVudCI6eyJlbnZpcm9ubWVudElkIjoiNWU3NGQwODYtZThhZS00YjVhLWI2YTctZWY2ZWYyZjQ4YWQ3Iiwid29ya3NwYWNlSWQiOiJhMWJiNThmNy1iYmE1LTQ3OTEtODVkMi05NDVmNzk5NzJkMWMifX19LCJuYmZvcm1hdCI6NCwibmJmb3JtYXRfbWlub3IiOjV9"
nb_payloadBase64 = base64.b64decode(nb_payload.encode("ascii")).decode("ascii").encode('utf-8')
fabric.create_notebook(name=nb_name,description=nb_description, notebook_content=nb_payloadBase64)

**NB: MaintenanceLh**

In [ ]:
# MaintenanceLh
nb_name = "MaintenanceLh"
nb_description = "Maintenance Notebook to Optimize and Vacuum Lakehouse tables"
nb_payload = "eyJjZWxscyI6W3siY2VsbF90eXBlIjoibWFya2Rvd24iLCJzb3VyY2UiOlsiIyMjIyAqKkRlZmluaW5nIFBhcmFtZXRlcnMqKiJdLCJtZXRhZGF0YSI6eyJudGVyYWN0Ijp7InRyYW5zaWVudCI6eyJkZWxldGluZyI6ZmFsc2V9fSwibWljcm9zb2Z0Ijp7Imxhbmd1YWdlIjoicHl0aG9uIiwibGFuZ3VhZ2VfZ3JvdXAiOiJzeW5hcHNlX3B5c3BhcmsifX0sImlkIjoiMDc3YWExY2QtNTcxNy00NjIwLWEzZTEtMmFkMzE2MjJmMjU0In0seyJjZWxsX3R5cGUiOiJjb2RlIiwic291cmNlIjpbIiMgUGFyYW1ldGVycyBjZWxsXG4iLCJsYWtlaG91c2VfbmFtZSA9IFwiTEgyXCIiXSwib3V0cHV0cyI6W3sib3V0cHV0X3R5cGUiOiJkaXNwbGF5X2RhdGEiLCJkYXRhIjp7ImFwcGxpY2F0aW9uL3ZuZC5saXZ5LnN0YXRlbWVudC1tZXRhK2pzb24iOnsic3BhcmtfcG9vbCI6bnVsbCwic3RhdGVtZW50X2lkIjotMSwic3RhdGVtZW50X2lkcyI6W10sInN0YXRlIjoiY2FuY2VsbGVkIiwibGl2eV9zdGF0ZW1lbnRfc3RhdGUiOm51bGwsInNlc3Npb25faWQiOm51bGwsIm5vcm1hbGl6ZWRfc3RhdGUiOiJjYW5jZWxsZWQiLCJxdWV1ZWRfdGltZSI6IjIwMjUtMDEtMDhUMTY6MTA6NDkuMjQxMTM0M1oiLCJzZXNzaW9uX3N0YXJ0X3RpbWUiOiIyMDI1LTAxLTA4VDE2OjEwOjQ5LjQ5NTI4MjlaIiwiZXhlY3V0aW9uX3N0YXJ0X3RpbWUiOm51bGwsImV4ZWN1dGlvbl9maW5pc2hfdGltZSI6IjIwMjUtMDEtMDhUMTY6MTA6NTIuMDY0ODk0M1oiLCJwYXJlbnRfbXNnX2lkIjoiMDE0Mjk1YWItODA4OS00OTdjLThmOWItYTVmMzM1M2ZjMWVjIn0sInRleHQvcGxhaW4iOiJTdGF0ZW1lbnRNZXRhKCwgLCAtMSwgQ2FuY2VsbGVkLCAsIENhbmNlbGxlZCkifSwibWV0YWRhdGEiOnt9fV0sImV4ZWN1dGlvbl9jb3VudCI6MSwibWV0YWRhdGEiOnsibWljcm9zb2Z0Ijp7Imxhbmd1YWdlIjoicHl0aG9uIiwibGFuZ3VhZ2VfZ3JvdXAiOiJzeW5hcHNlX3B5c3BhcmsifSwidGFncyI6WyJwYXJhbWV0ZXJzIl19LCJpZCI6IjQyMjQ0NDNmLWMyZTItNDhkNS1hOTZkLTZkY2QxN2Q5NmZjZiJ9LHsiY2VsbF90eXBlIjoiY29kZSIsInNvdXJjZSI6WyIjIyBBbGwgY3JlZGl0IGFuZCBzaG91dG91dCB0byB0aGUgZm9sbG93aW5nIGJsb2dcbiIsIiMjaHR0cHM6Ly9tZWRpdW0uY29tL0BncmVnYS5ocmVuL2NyZWF0aW5nLXJldXNhYmxlLXNwYXJrLW5vdGVib29rcy1pbi1taWNyb3NvZnQtZmFicmljLWU2MzJjMTgyNDgwOCJdLCJvdXRwdXRzIjpbeyJvdXRwdXRfdHlwZSI6ImRpc3BsYXlfZGF0YSIsImRhdGEiOnsiYXBwbGljYXRpb24vdm5kLmxpdnkuc3RhdGVtZW50LW1ldGEranNvbiI6eyJzcGFya19wb29sIjpudWxsLCJzdGF0ZW1lbnRfaWQiOm51bGwsInN0YXRlbWVudF9pZHMiOm51bGwsInN0YXRlIjoiY2FuY2VsbGVkIiwibGl2eV9zdGF0ZW1lbnRfc3RhdGUiOm51bGwsInNlc3Npb25faWQiOm51bGwsIm5vcm1hbGl6ZWRfc3RhdGUiOiJjYW5jZWxsZWQiLCJxdWV1ZWRfdGltZSI6IjIwMjUtMDEtMDhUMTY6MTA6NDkuMjQ0NTA1NloiLCJzZXNzaW9uX3N0YXJ0X3RpbWUiOm51bGwsImV4ZWN1dGlvbl9zdGFydF90aW1lIjpudWxsLCJleGVjdXRpb25fZmluaXNoX3RpbWUiOiIyMDI1LTAxLTA4VDE2OjEwOjUyLjA2MjQ2MDVaIiwicGFyZW50X21zZ19pZCI6IjY4NWVjZDJjLTkyYjAtNDE4MC04ODI5LWJkYWQ2OTczYTc3ZCJ9LCJ0ZXh0L3BsYWluIjoiU3RhdGVtZW50TWV0YSgsICwgLCBDYW5jZWxsZWQsICwgQ2FuY2VsbGVkKSJ9LCJtZXRhZGF0YSI6e319XSwiZXhlY3V0aW9uX2NvdW50IjpudWxsLCJtZXRhZGF0YSI6eyJqdXB5dGVyIjp7InNvdXJjZV9oaWRkZW4iOmZhbHNlLCJvdXRwdXRzX2hpZGRlbiI6ZmFsc2V9LCJudGVyYWN0Ijp7InRyYW5zaWVudCI6eyJkZWxldGluZyI6ZmFsc2V9fSwibWljcm9zb2Z0Ijp7Imxhbmd1YWdlIjoicHl0aG9uIiwibGFuZ3VhZ2VfZ3JvdXAiOiJzeW5hcHNlX3B5c3BhcmsifSwiZWRpdGFibGUiOmZhbHNlLCJydW5fY29udHJvbCI6eyJmcm96ZW4iOnRydWV9fSwiaWQiOiIyMjNkZmE4MC02Yjk2LTRlOWQtOWUzYy0xY2UzMDczZGZhYmQifSx7ImNlbGxfdHlwZSI6Im1hcmtkb3duIiwic291cmNlIjpbIiMjIyMgKipNb3VudGluZyB0aGUgTGFrZWhvdXNlKioiXSwibWV0YWRhdGEiOnsibnRlcmFjdCI6eyJ0cmFuc2llbnQiOnsiZGVsZXRpbmciOmZhbHNlfX0sIm1pY3Jvc29mdCI6eyJsYW5ndWFnZSI6InB5dGhvbiIsImxhbmd1YWdlX2dyb3VwIjoic3luYXBzZV9weXNwYXJrIn19LCJpZCI6ImFkZjBhZWNiLWQzYjYtNDJjNS1hYzQ1LTc2OTAwY2IxZTM5YiJ9LHsiY2VsbF90eXBlIjoiY29kZSIsInNvdXJjZSI6WyIjIE1vdW50IHRoZSBsYWtlaG91c2VcbiIsImxha2Vob3VzZSA9IG1zc3Bhcmt1dGlscy5sYWtlaG91c2UuZ2V0KGxha2Vob3VzZV9uYW1lKVxuIiwibXNzcGFya3V0aWxzLmZzLm1vdW50KGxha2Vob3VzZS5nZXQoXCJwcm9wZXJ0aWVzXCIpLmdldChcImFiZnNQYXRoXCIpLCBcIi9vcHRpbWl6ZV9sYWtlaG91c2VcIilcbiIsIiMgR2V0IHRoZSBsb2NhbCBwYXRoIHdoZXJlIHRoZSBsYWtlaG91c2UgaXMgbW91bnRlZFxuIiwicGF0aCA9IG1zc3Bhcmt1dGlscy5mcy5nZXRNb3VudFBhdGgoXCIvb3B0aW1pemVfbGFrZWhvdXNlXCIpIl0sIm91dHB1dHMiOlt7Im91dHB1dF90eXBlIjoiZGlzcGxheV9kYXRhIiwiZGF0YSI6eyJhcHBsaWNhdGlvbi92bmQubGl2eS5zdGF0ZW1lbnQtbWV0YStqc29uIjp7InNwYXJrX3Bvb2wiOm51bGwsInN0YXRlbWVudF9pZCI6bnVsbCwic3RhdGVtZW50X2lkcyI6bnVsbCwic3RhdGUiOiJjYW5jZWxsZWQiLCJsaXZ5X3N0YXRlbWVudF9zdGF0ZSI6bnVsbCwic2Vzc2lvbl9pZCI6bnVsbCwibm9ybWFsaXplZF9zdGF0ZSI6ImNhbmNlbGxlZCIsInF1ZXVlZF90aW1lIjoiMjAyNS0wMS0wOFQxNjoxMDo0OS4yNDUzNzcxWiIsInNlc3Npb25fc3RhcnRfdGltZSI6bnVsbCwiZXhlY3V0aW9uX3N0YXJ0X3RpbWUiOm51bGwsImV4ZWN1dGlvbl9maW5pc2hfdGltZSI6IjIwMjUtMDEtMDhUMTY6MTA6NTIuMDYzMTIxOFoiLCJwYXJlbnRfbXNnX2lkIjoiM2MwNjYxYmItMDI4Yy00N2VjLWE0MzEtZjQxYWVmMDBiMGUxIn0sInRleHQvcGxhaW4iOiJTdGF0ZW1lbnRNZXRhKCwgLCAsIENhbmNlbGxlZCwgLCBDYW5jZWxsZWQpIn0sIm1ldGFkYXRhIjp7fX1dLCJleGVjdXRpb25fY291bnQiOm51bGwsIm1ldGFkYXRhIjp7Imp1cHl0ZXIiOnsic291cmNlX2hpZGRlbiI6ZmFsc2UsIm91dHB1dHNfaGlkZGVuIjpmYWxzZX0sIm50ZXJhY3QiOnsidHJhbnNpZW50Ijp7ImRlbGV0aW5nIjpmYWxzZX19LCJtaWNyb3NvZnQiOnsibGFuZ3VhZ2UiOiJweXRob24iLCJsYW5ndWFnZV9ncm91cCI6InN5bmFwc2VfcHlzcGFyayJ9LCJjb2xsYXBzZWQiOmZhbHNlfSwiaWQiOiJhODg2MDk2MC03MzdhLTQ1Y2UtYjIzZi04Mjk4YjNhNjAzMWUifSx7ImNlbGxfdHlwZSI6ImNvZGUiLCJzb3VyY2UiOlsiIyBSZXRyaWV2ZSBhbGwgdGFibGUgbmFtZXNcbiIsImltcG9ydCBvc1xuIiwidGFibGVzID0gb3MubGlzdGRpcihmXCJ7cGF0aH0vVGFibGVzXCIpIl0sIm91dHB1dHMiOlt7Im91dHB1dF90eXBlIjoiZGlzcGxheV9kYXRhIiwiZGF0YSI6eyJhcHBsaWNhdGlvbi92bmQubGl2eS5zdGF0ZW1lbnQtbWV0YStqc29uIjp7InNwYXJrX3Bvb2wiOm51bGwsInN0YXRlbWVudF9pZCI6bnVsbCwic3RhdGVtZW50X2lkcyI6bnVsbCwic3RhdGUiOiJjYW5jZWxsZWQiLCJsaXZ5X3N0YXRlbWVudF9zdGF0ZSI6bnVsbCwic2Vzc2lvbl9pZCI6bnVsbCwibm9ybWFsaXplZF9zdGF0ZSI6ImNhbmNlbGxlZCIsInF1ZXVlZF90aW1lIjoiMjAyNS0wMS0wOFQxNjoxMDo0OS4yNDYxNTY2WiIsInNlc3Npb25fc3RhcnRfdGltZSI6bnVsbCwiZXhlY3V0aW9uX3N0YXJ0X3RpbWUiOm51bGwsImV4ZWN1dGlvbl9maW5pc2hfdGltZSI6IjIwMjUtMDEtMDhUMTY6MTA6NTIuMDYzNjIxNFoiLCJwYXJlbnRfbXNnX2lkIjoiNDc3MzVlODUtNTBiYy00NDQ0LWI4NzQtOWZiMzU1ODMzMGU3In0sInRleHQvcGxhaW4iOiJTdGF0ZW1lbnRNZXRhKCwgLCAsIENhbmNlbGxlZCwgLCBDYW5jZWxsZWQpIn0sIm1ldGFkYXRhIjp7fX1dLCJleGVjdXRpb25fY291bnQiOm51bGwsIm1ldGFkYXRhIjp7Imp1cHl0ZXIiOnsic291cmNlX2hpZGRlbiI6ZmFsc2UsIm91dHB1dHNfaGlkZGVuIjpmYWxzZX0sIm50ZXJhY3QiOnsidHJhbnNpZW50Ijp7ImRlbGV0aW5nIjpmYWxzZX19LCJtaWNyb3NvZnQiOnsibGFuZ3VhZ2UiOiJweXRob24iLCJsYW5ndWFnZV9ncm91cCI6InN5bmFwc2VfcHlzcGFyayJ9fSwiaWQiOiJjZjA3MTVjYy03NWMxLTRlMmQtYmUwOS1mY2ZlY2VlZDc4YWIifSx7ImNlbGxfdHlwZSI6Im1hcmtkb3duIiwic291cmNlIjpbIiMjIyMgKipWYWN1dW0gYW5kIE9wdGltaXplIE9wZXJhdGlvbnMqKiJdLCJtZXRhZGF0YSI6eyJudGVyYWN0Ijp7InRyYW5zaWVudCI6eyJkZWxldGluZyI6ZmFsc2V9fSwibWljcm9zb2Z0Ijp7Imxhbmd1YWdlIjoicHl0aG9uIiwibGFuZ3VhZ2VfZ3JvdXAiOiJzeW5hcHNlX3B5c3BhcmsifX0sImlkIjoiYTcyZDYwYTUtZWIxNS00NzcyLWFlZDUtMjhmNjY2M2EwMjU4In0seyJjZWxsX3R5cGUiOiJjb2RlIiwic291cmNlIjpbImZyb20gZGVsdGEudGFibGVzIGltcG9ydCBEZWx0YVRhYmxlXG4iLCJcbiIsIiMgVmFjdXVtIGFuZCBvcHRpbWl6ZSBlYWNoIHRhYmxlXG4iLCJmb3IgdGFibGVfbmFtZSBpbiB0YWJsZXM6XG4iLCIgICAgZGVsdGFfdGFibGUgPSBEZWx0YVRhYmxlLmZvclBhdGgoc3BhcmssIGZcIi97bGFrZWhvdXNlLmdldCgnaWQnKX0vVGFibGVzL3t0YWJsZV9uYW1lfVwiKVxuIiwiICAgIGRlbHRhX3RhYmxlLnZhY3V1bSgpXG4iLCIgICAgZGVsdGFfdGFibGUub3B0aW1pemUoKS5leGVjdXRlQ29tcGFjdGlvbigpIl0sIm91dHB1dHMiOltdLCJleGVjdXRpb25fY291bnQiOm51bGwsIm1ldGFkYXRhIjp7Imp1cHl0ZXIiOnsic291cmNlX2hpZGRlbiI6ZmFsc2UsIm91dHB1dHNfaGlkZGVuIjpmYWxzZX0sIm50ZXJhY3QiOnsidHJhbnNpZW50Ijp7ImRlbGV0aW5nIjpmYWxzZX19LCJtaWNyb3NvZnQiOnsibGFuZ3VhZ2UiOiJweXRob24iLCJsYW5ndWFnZV9ncm91cCI6InN5bmFwc2VfcHlzcGFyayJ9LCJlZGl0YWJsZSI6dHJ1ZSwicnVuX2NvbnRyb2wiOnsiZnJvemVuIjpmYWxzZX19LCJpZCI6ImIxZDA0MWIyLTU0MjQtNDczNC1iMDk4LTNjZDY1N2YwZjJhZiJ9XSwibWV0YWRhdGEiOnsia2VybmVsX2luZm8iOnsibmFtZSI6InN5bmFwc2VfcHlzcGFyayJ9LCJrZXJuZWxzcGVjIjp7Im5hbWUiOiJzeW5hcHNlX3B5c3BhcmsiLCJkaXNwbGF5X25hbWUiOiJzeW5hcHNlX3B5c3BhcmsifSwibGFuZ3VhZ2VfaW5mbyI6eyJuYW1lIjoicHl0aG9uIn0sIm1pY3Jvc29mdCI6eyJsYW5ndWFnZSI6InB5dGhvbiIsImxhbmd1YWdlX2dyb3VwIjoic3luYXBzZV9weXNwYXJrIiwibXNfc3BlbGxfY2hlY2siOnsibXNfc3BlbGxfY2hlY2tfbGFuZ3VhZ2UiOiJlbiJ9fSwibnRlcmFjdCI6eyJ2ZXJzaW9uIjoibnRlcmFjdC1mcm9udC1lbmRAMS4wLjAifSwic3luYXBzZV93aWRnZXQiOnsidmVyc2lvbiI6IjAuMSIsInN0YXRlIjp7fX0sInNwYXJrX2NvbXB1dGUiOnsiY29tcHV0ZV9pZCI6Ii90cmlkZW50L2RlZmF1bHQiLCJzZXNzaW9uX29wdGlvbnMiOnsiY29uZiI6eyJzcGFyay5zeW5hcHNlLm5icy5zZXNzaW9uLnRpbWVvdXQiOiIxMjAwMDAwIn19fSwiZGVwZW5kZW5jaWVzIjp7Imxha2Vob3VzZSI6eyJkZWZhdWx0X2xha2Vob3VzZSI6IjgyZTE2ODY4LTFhMWEtNGMzMy04NTlmLTg3MmY5MzYzMTVhZCIsImRlZmF1bHRfbGFrZWhvdXNlX25hbWUiOiJsaF9Gbk8iLCJkZWZhdWx0X2xha2Vob3VzZV93b3Jrc3BhY2VfaWQiOiJhMWJiNThmNy1iYmE1LTQ3OTEtODVkMi05NDVmNzk5NzJkMWMifX19LCJuYmZvcm1hdCI6NCwibmJmb3JtYXRfbWlub3IiOjV9"
nb_payloadBase64 = base64.b64decode(nb_payload.encode("ascii")).decode("ascii").encode('utf-8')
fabric.create_notebook(name=nb_name,description=nb_description, notebook_content=nb_payloadBase64)

#### **Lakehouses**

**LH2**

In [ ]:
# LH2
workspaceId = fabric2.get_workspace_id()
url = f"v1/workspaces/{workspaceId}/items"
payload = {"displayName": "LH2","type": "Lakehouse"}
result = client.post(url,json=payload)


**LH3**

In [ ]:
# LH3
workspaceId = fabric2.get_workspace_id()
url = f"v1/workspaces/{workspaceId}/items"
payload = {"displayName": "LH3","type": "Lakehouse"}
result = client.post(url,json=payload)


#### **Warehouse**

**WH: WatermarkWarehouse**

In [ ]:
# WatermarkWarehouse
warehouse_name="WatermarkWarehouse"
workspaceId = fabric2.get_workspace_id()
url = f"v1/workspaces/{workspaceId}/items"
payload = {"displayName": "" + warehouse_name + "","type": "Warehouse"}
result = client.post(url,json=payload)

**Insert initial data into Warehouse**

In [ ]:
# wait 10 seconds until above warehouse has been created and is available in API
time.sleep(10)

# get server connection string
workspaceId = fabric2.get_workspace_id()
url = f"v1/workspaces/{workspaceId}/warehouses"
result = client.get(url)
df_items = pd.json_normalize(result.json()['value'])
filtereddf = df_items[df_items['displayName'] == warehouse_name]
server = filtereddf['properties.connectionString']

#Instantiate sql connection through pyodbc
connection_string = f"Driver={{ODBC Driver 18 for SQL Server}};Server={server}"
engine = create_engine(connection_string)

sqlStatement = '-- create watermark table\
CREATE TABLE WatermarkWarehouse.watermarktable\
(\
WatermarkValue VARCHAR(255)\
); \
\
-- Create usp for watermark update\
CREATE PROCEDURE WatermarkWarehouse.usp_write_watermark @LastFolderName varchar(255)\
AS\
BEGIN\
  UPDATE watermarktable\
  SET [WatermarkValue] = @LastFolderName\
END; \
\
-- Insert empty watermark record\
INSERT INTO WatermarkWarehouse.watermarktable ([WatermarkValue]) VALUES ( '' );'

with engine.connect() as alchemy_connection:    
    connection = engine.raw_connection()
    cursor = connection.cursor()    
    cursor.execute(sqlStatement)
    connection.commit()

#### **Pipelines**

**Pipeline: CDMtoLh**

In [ ]:
# CDMtoLh
pipe_name = "CDMtoLh"
pipe_description = "Incremental Data Pipeline from CDM to Lakehouse"
fabric.create_data_pipeline(name=pipe_name, description=pipe_description)

**Pipeline: SparkSessionKeepAlive**

In [ ]:
# SparkSessionKeepAlive
pipe_name = "SparkSessionKeepAlive"
pipe_description = "Spark Session Keep Alive Pipeline including Session Tag"
fabric.create_data_pipeline(name=pipe_name, description=pipe_description)

#### **Environments**

**Environment: CDMtoLh**

In [ ]:
# CDMtoLh
env_name = "CDMtoLh"
env_description = "Environment for CDM to Lakehouse"
fabric.create_environment(environment=env_name, description=env_description)